In [1]:
from collections import defaultdict, Counter
import datetime as dt
import emoji
from icecream import ic
from IPython.display import display_html 
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import regex as rex
import shutil
from string import punctuation
import time
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords

import spacy

from sklearn.feature_extraction.text import TfidfTransformer, \
CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline

import textacy.preprocessing as tprep
from textacy.extract import keyword_in_context

In [2]:
to_csv_flag = False
# Establish working directories for saving dataframes as CSV files.

'''Dir nav citation:
https://softhints.com/python-change-directory-parent/
'''
curr_dir = os.path.abspath(os.curdir)
print(curr_dir)
os.chdir("..")
up1_dir = os.path.abspath(os.curdir)
print(up1_dir)

C:\Users\acarr\Documents\GitHub\599_team_project\deliverables
C:\Users\acarr\Documents\GitHub\599_team_project


In [3]:
# Get current date/time to append to file name string.

today = dt.datetime.today()
today= str(today)
today = today.replace(':', '-').replace('.', '').replace(' ', '_')
print(today)
print(type(today))

2023-07-29_13-59-18493660
<class 'str'>


In [4]:
# change `data_large_location` to the location of the folder on your machine.
data_large_location = 'data_large'
ref_docs_location = 'ref_docs'

file_in_name01 = 'news-04.csv'

file_in_path01 = os.path.join(up1_dir, data_large_location, file_in_name01)

print(f'CSV file in 1 path: {file_in_path01}')

CSV file in 1 path: C:\Users\acarr\Documents\GitHub\599_team_project\data_large\news-04.csv


### Review dataframe

Read in data from CSV, check resulting dataframe shape, and display first several records.

In [5]:
slct_tbl_full_df01 = pd.read_csv(file_in_path01)

In [6]:
print(f'Dataframe shape: {slct_tbl_full_df01.shape}')
display(slct_tbl_full_df01.head())

Dataframe shape: (36405, 8)


,text_id,sentiment_vader,sentiment_vader_compound,sentiment_vader_cat,sentiment_bert_prob,sentiment_bert_cat,sentiment_bert,embeddings
0,2,"{'neg': 0.041, 'neu': 0.888, 'pos': 0.072, 'co...",0.8658,positive,0.009480,negative,-0.981040,"tensor([-7.5552e-02, 5.0567e-03, -4.7665e-02,..."
1,3,"{'neg': 0.043, 'neu': 0.85, 'pos': 0.107, 'com...",0.9769,positive,0.981083,positive,0.962166,"tensor([ 4.0231e-02, -1.2047e-02, 7.2875e-02,..."
2,5,"{'neg': 0.02, 'neu': 0.817, 'pos': 0.163, 'com...",0.9900,positive,0.013669,negative,-0.972662,"tensor([-1.1686e-01, -1.0624e-01, -7.8918e-03,..."
3,6,"{'neg': 0.016, 'neu': 0.888, 'pos': 0.096, 'co...",0.9788,positive,0.993403,positive,0.986807,"tensor([-9.5328e-03, 4.5495e-03, -9.8373e-02,..."
4,7,"{'neg': 0.061, 'neu': 0.853, 'pos': 0.085, 'co...",0.6697,positive,0.910206,positive,0.820413,"tensor([-2.5785e-02, 7.9342e-02, -2.9590e-02,..."


In [14]:
display(slct_tbl_full_df01.describe())

,text_id,sentiment_vader_compound,sentiment_bert_prob,sentiment_bert
count,36405.000000,36405.000000,36405.000000,36405.000000
mean,22393.006867,0.388630,0.252910,-0.494180
std,16159.115108,0.842946,0.388436,0.776872
min,2.000000,-1.000000,0.000201,-0.999597
25%,10616.000000,-0.739100,0.003759,-0.992482
50%,20320.000000,0.963800,0.013805,-0.972389
75%,29543.000000,0.995600,0.464821,-0.070359
max,71592.000000,1.000000,0.999794,0.999587


In [15]:
slct_tbl_full_df01a = slct_tbl_full_df01.loc[slct_tbl_full_df01['sentiment_bert'] > .95]

In [16]:
print(f'Dataframe shape: {slct_tbl_full_df01a.shape}')
display(slct_tbl_full_df01a.head())

Dataframe shape: (4617, 8)


,text_id,sentiment_vader,sentiment_vader_compound,sentiment_vader_cat,sentiment_bert_prob,sentiment_bert_cat,sentiment_bert,embeddings
1,3,"{'neg': 0.043, 'neu': 0.85, 'pos': 0.107, 'com...",0.9769,positive,0.981083,positive,0.962166,"tensor([ 4.0231e-02, -1.2047e-02, 7.2875e-02,..."
3,6,"{'neg': 0.016, 'neu': 0.888, 'pos': 0.096, 'co...",0.9788,positive,0.993403,positive,0.986807,"tensor([-9.5328e-03, 4.5495e-03, -9.8373e-02,..."
10,23,"{'neg': 0.008, 'neu': 0.772, 'pos': 0.219, 'co...",0.9898,positive,0.982865,positive,0.965731,"tensor([-8.2832e-02, 3.1004e-02, 2.1707e-02,..."
16,33,"{'neg': 0.029, 'neu': 0.767, 'pos': 0.204, 'co...",0.9998,positive,0.995310,positive,0.990621,"tensor([-3.9029e-02, -9.3190e-02, 5.2567e-02,..."
22,45,"{'neg': 0.0, 'neu': 0.852, 'pos': 0.148, 'comp...",0.9910,positive,0.997603,positive,0.995207,"tensor([-4.3832e-02, -8.0221e-03, -7.6465e-03,..."


In [9]:
# Set path to write CSV file to.

file_out_name01 = f'data_sa_w_sw_half1_{today}.csv'
file_out_name02 = f'data_sa_w_sw_half2_{today}.csv'

file_out_path01 = os.path.join(up1_dir, file_out_name01)
file_out_path02 = os.path.join(up1_dir, file_out_name02)

print(f'CSV file out 1 path: {file_out_path01}')
print(f'CSV file out 2 path: {file_out_path02}')

export_col_names_lst = ['text_id',
                        'sentiment_bert',
                       ]

# Write pandas dataframe to CSV; save locally.

len_half = int(round(len(slct_tbl_full_df01)/2,0))
print(len_half)

if to_csv_flag == False:
    pass
else:
    slct_tbl_full_df01[export_col_names_lst][:len_half]\
    .to_csv(file_out_path01, index=False)
    slct_tbl_full_df01[export_col_names_lst][len_half:]\
    .to_csv(file_out_path02, index=False)

print(slct_tbl_full_df01.shape)
display(slct_tbl_full_df01.sample(5))

CSV file out 1 path: C:\Users\acarr\Documents\GitHub\599_team_project\data_sa_w_sw_half1_2023-07-29_13-59-18493660.csv
CSV file out 2 path: C:\Users\acarr\Documents\GitHub\599_team_project\data_sa_w_sw_half2_2023-07-29_13-59-18493660.csv
18202
(36405, 8)


,text_id,sentiment_vader,sentiment_vader_compound,sentiment_vader_cat,sentiment_bert_prob,sentiment_bert_cat,sentiment_bert,embeddings
10820,12431,"{'neg': 0.051, 'neu': 0.735, 'pos': 0.215, 'co...",0.9989,positive,0.009596,negative,-0.980807,"tensor([-8.2225e-02, -8.2994e-03, -3.3327e-02,..."
15645,17730,"{'neg': 0.037, 'neu': 0.835, 'pos': 0.128, 'co...",0.9863,positive,0.630654,positive,0.261307,"tensor([-1.0458e-01, -3.1580e-03, -3.5578e-02,..."
28240,30497,"{'neg': 0.113, 'neu': 0.846, 'pos': 0.041, 'co...",-0.9931,negative,0.000531,negative,-0.998938,"tensor([-2.3565e-02, -1.8127e-02, 9.8111e-05,..."
13248,15242,"{'neg': 0.084, 'neu': 0.79, 'pos': 0.126, 'com...",0.9619,positive,0.001638,negative,-0.996724,"tensor([-3.9353e-02, 4.6926e-02, -1.9880e-03,..."
27838,30091,"{'neg': 0.023, 'neu': 0.854, 'pos': 0.124, 'co...",0.9969,positive,0.118642,negative,-0.762716,"tensor([-6.8907e-02, -7.9617e-03, 2.8703e-02,..."
